In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Dataset.csv')
df.head()

,Date,Time_in,Time_in.1,National_holiday
0,01-08-2024,01-08-2024 06:00,01-08-2024 15:00,0.0
1,02-08-2024,02-08-2024 08:00,02-08-2024 19:00,0.0
2,03-08-2024,03-08-2024 18:00,04-08-2024 10:00,0.0
3,04-08-2024,04-08-2024 11:00,04-08-2024 21:00,0.0
4,05-08-2024,05-08-2024 07:00,05-08-2024 15:00,0.0


In [4]:
df.rename(columns={'Time_in.1' : 'time_out'}, inplace = True)

In [5]:
df.rename(columns={'Time_in' : 'time_in'}, inplace = True)

In [6]:
df['time_in'] = pd.to_datetime(df['time_in'], format='%d-%m-%Y %H:%M')

df['time_out'] = pd.to_datetime(df['time_out'], format='%d-%m-%Y %H:%M')

In [7]:
df.dtypes

Date                        object
time_in             datetime64[ns]
time_out            datetime64[ns]
National_holiday           float64
dtype: object

In [8]:
df.head()

,Date,time_in,time_out,National_holiday
0,01-08-2024,2024-08-01 06:00:00,2024-08-01 15:00:00,0.0
1,02-08-2024,2024-08-02 08:00:00,2024-08-02 19:00:00,0.0
2,03-08-2024,2024-08-03 18:00:00,2024-08-04 10:00:00,0.0
3,04-08-2024,2024-08-04 11:00:00,2024-08-04 21:00:00,0.0
4,05-08-2024,2024-08-05 07:00:00,2024-08-05 15:00:00,0.0


In [14]:
def calculate_daily_wage(row):
    working_hours = (row['time_out'] - row['time_in']) / pd.Timedelta(hours=1)
    daily_wage = 0  # To store the daily wage for regular hours and overtime from 2 to 5 am

    if row['National_holiday']:  # Check for national holiday
        return [working_hours, working_hours * 130, 0]
    else:
        if working_hours <= 9:
            return [working_hours, working_hours * 100, 0]
        else:
            daily_wage = 900
            overtime_hours = 0  # To store overtime hours excluding 2 to 5 am
            regular_off_time = row['time_in'] + pd.Timedelta(hours=9)  # Regular duty off time
            next_day = row['time_in'].date() + pd.Timedelta(days=1)

            if regular_off_time.date() == next_day:
                if regular_off_time.hour <= 2:
                    if row['time_out'].hour >= 5:
                        daily_wage += 3 * 125  # Extra pay for working from 2 AM to 5 AM
                        overtime_hours = (row['time_out'].replace(hour=2, minute=0, second=0) - regular_off_time) / pd.Timedelta(hours=1)
                    else:
                        if row['time_out'].hour <= 2:
                            overtime_hours = (row['time_out'] - regular_off_time) / pd.Timedelta(hours=1)
                        else:
                            overtime_hours = (row['time_out'].replace(hour=2, minute=0, second=0) - regular_off_time) / pd.Timedelta(hours=1)
                            daily_wage += (row['time_out'] - row['time_in'].replace(hour=2, minute=0, second=0)) / pd.Timedelta(hours=1) * 125

                elif 2 < regular_off_time.hour <= 5:
                    if row['time_out'].hour <= 5:
                        daily_wage += (row['time_out'] - regular_off_time) / pd.Timedelta(hours=1) * 125
                    else:
                        daily_wage += (regular_off_time.replace(hour=5, minute=0, second=0) - regular_off_time) / pd.Timedelta(hours=1) * 125
                        overtime_hours = (row['time_out'] - regular_off_time.replace(hour=5, minute=0, second=0)) / pd.Timedelta(hours=1)
                else:
                    overtime_hours = (row['time_out'] - regular_off_time) / pd.Timedelta(hours=1)
            else:
                overtime_hours = (row['time_out'] - regular_off_time) / pd.Timedelta(hours=1)

    return [working_hours, daily_wage, overtime_hours]

In [15]:
df[['total_work_hours', 'amount', 'extra_hours']] = df.apply(func2, axis=1, result_type='expand')

In [16]:
df.head(10)

,Date,time_in,time_out,National_holiday,total_work_hours,amount,extra_hours
0,01-08-2024,2024-08-01 06:00:00,2024-08-01 15:00:00,0.0,9.0,900.0,0.0
1,02-08-2024,2024-08-02 08:00:00,2024-08-02 19:00:00,0.0,11.0,900.0,2.0
2,03-08-2024,2024-08-03 18:00:00,2024-08-04 10:00:00,0.0,16.0,1150.0,5.0
3,04-08-2024,2024-08-04 11:00:00,2024-08-04 21:00:00,0.0,10.0,900.0,1.0
4,05-08-2024,2024-08-05 07:00:00,2024-08-05 15:00:00,0.0,8.0,800.0,0.0
5,07-08-2024,2024-08-07 11:00:00,2024-08-07 21:00:00,1.0,10.0,1300.0,0.0
6,08-08-2024,2024-08-08 00:00:00,2024-08-08 08:00:00,0.0,8.0,800.0,0.0
7,09-08-2024,2024-08-09 20:00:00,2024-08-10 09:00:00,0.0,13.0,900.0,4.0
8,10-08-2024,2024-08-10 07:00:00,2024-08-10 15:00:00,0.0,8.0,800.0,0.0
9,11-08-2024,2024-08-11 08:00:00,2024-08-11 16:00:00,0.0,8.0,800.0,0.0


**The paycheck amount for the given test case is calculated as follows:**

**Week 1:**

Daily duty hours wage: 900 + 900 + 1150 + 900 + 800

Overtime hours wage (2 to 5 am): 8 * 115

Total for Week 1: 5570

**National Holiday:**

National holiday wage: 1300

**Week 2:**

Daily duty hours wage: 800 + 900 + 800 + 800 + 800

Overtime hours wage (2 to 5 am): 4 * 120

Total for Week 2: 4580

**Overall Total:**

Sum of Week 1, national holiday, and Week 2 wages: 11450

Note: The 'amount' column only contains the daily duty hours wage and the overtime hours wage for hours between 2 to 5 am.

To calculate the final paycheck amount, a function can be created to add up the weekly work hours and determine the correct payoff rate for overtime (excluding 2 to 5 am hours, as their wage is already included in the "amount").

**For choosing the overtime rate:**

Overtime hours = max(0, total_hours - 45) (subtracting the regular 45 hours per week)
Overtime rate: 1.2 if overtime hours are less than or equal to 5, otherwise 1.15
